In [1]:
import os
import random
import numpy as np
import pandas as pd 
from skimage import io
from skimage import color
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from dask.array.image import imread
from dask import bag, threaded
from dask.diagnostics import ProgressBar
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import pickle
from skimage.transform import resize
from sys import getsizeof

import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.autograd import Variable
import torch.nn.functional as F

In [2]:
driver_details = pd.read_csv('imgs/driver_imgs_list.csv',na_values='na')
print(driver_details.head(5))

  subject classname            img
0    p002        c0  img_44733.jpg
1    p002        c0  img_72999.jpg
2    p002        c0  img_25094.jpg
3    p002        c0  img_69092.jpg
4    p002        c0  img_92629.jpg


In [3]:
train_image = []
for i in range(10):
    print('now we are in the folder C',i)
    imgs = os.listdir("imgs/train/c"+str(i))
    print(len(imgs))
    for j in range(len(imgs)): 
        img_name = "imgs/train/c"+str(i)+"/"+imgs[j]
        img = imread(img_name)
        img = color.rgb2gray(img[0])
        img = resize(img,(128,128))
        label = i
        driver = driver_details[driver_details['img'] == imgs[j]]['subject'].values[0]
        train_image.append([img,label,driver])

now we are in the folder C 0
2489
now we are in the folder C 1
2267
now we are in the folder C 2
2317
now we are in the folder C 3
2346
now we are in the folder C 4
2326
now we are in the folder C 5
2312
now we are in the folder C 6
2325
now we are in the folder C 7
2002
now we are in the folder C 8
1911
now we are in the folder C 9
2129


In [22]:
## getting list of driver names
D = []
for features,labels,drivers in train_image:
    D.append(drivers)
## Deduplicating drivers
deduped = []
for i in D:
    if i not in deduped:
        deduped.append(i)   
## selecting random drivers for the validation set
driv_selected = []
import random
driv_nums = random.sample(range(len(deduped)), 8)
for i in driv_nums:
    driv_selected.append(deduped[i])

In [23]:
## Splitting the train and val
X_train= []
y_train = []
X_val = []
y_val = []
for features,labels,drivers in train_image:
    if drivers in driv_selected:
        X_val.append(features)
        y_val.append(labels)   
    else:
        X_train.append(features)
        y_train.append(labels)yy

In [25]:
X_train = np.array(X_train).reshape(-1,128,128,1)
X_val = np.array(X_val).reshape(-1,128,128,1)

y_train=np.array(y_train)
y_val=np.array(y_val)

In [26]:
unlab_image = []

print('now we are in the test folder')
imgs = os.listdir("imgs/test")
for j in range(40000):
    img_name = "imgs/test/"+imgs[j]
    img = imread(img_name)
    img = color.rgb2gray(img[0])
    img = resize(img,(128,128))
    label = -1
    unlab_image.append([img,label])

now we are in the test folder


In [27]:
X_unlab= []
y_unlab = []

for features,labels in unlab_image:
    X_unlab.append(features)
    y_unlab.append(labels)
    
print (len(X_unlab),len(y_unlab))

40000 40000


In [28]:
X_unlab = np.array(X_unlab).reshape(-1,128,128,1)
y_unlab = np.array(y_unlab)
X_unlab.shape,y_unlab.shape

((40000, 128, 128, 1), (40000,))